In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import os
# 1. 데이터 로드 (테스트용 데이터 프레임 생성)
# 실제 사용 시에는 pd.read_csv 사용
df = pd.read_csv('아파트_수집_최종.csv')
# data = {
#     'kaptCode': ['A10023990'],
#     'kaptName': ['청년주택 와이엔타워'],
#     'doroJuso': ['서울특별시 노원구 공릉동 동일로 1000']
# }
# df = pd.DataFrame(data)

# 2. 브라우저 옵션 강화 (봇 탐지 회피 및 안정성 확보)
chrome_options = Options()

TEMP_DATA_PATH = os.path.join(os.getcwd(), "temp_profile") 

# 프로필 폴더가 없으면 생성
if not os.path.exists(TEMP_DATA_PATH):
    os.makedirs(TEMP_DATA_PATH)
    print(f"새 임시 프로필 폴더 생성: {TEMP_DATA_PATH}")

chrome_options.add_argument(f"--user-data-dir={TEMP_DATA_PATH}")
# chrome_options.add_argument("--headless") # 디버깅 중에는 주석 처리하세요! (화면을 봐야 함)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080") # 화면 크기 고정 (반응형 레이아웃 문제 방지)
chrome_options.add_argument("lang=ko_KR")

# 봇 탐지 방지용 핵심 옵션
# chrome_options.add_argument("--disable-blink-features=AutomationControlled") 
# chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
# chrome_options.add_experimental_option("useAutomationExtension", False)

# User-Agent 변경
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    print("✅ 드라이버 세션 생성 성공!")
except Exception as e:
    print(f"❌ 드라이버 세션 생성 최종 실패: {e}")
    # 실패 시 생성된 임시 폴더 삭제 (선택 사항)
    # os.rmdir(TEMP_DATA_PATH)
    # print("실패하여 임시 프로필 폴더 삭제")
    raise e

# -------------------------------------------------------------
# [추가할 코드] 로그인 대기
# -------------------------------------------------------------
# 🚨 잠시 멈춤: 사용자가 수동으로 로그인할 시간을 줍니다.
print("\n🚨🚨 잠시 멈춥니다. 새 창에 Zippoom 로그인 후 콘솔에서 엔터를 눌러주세요. 🚨🚨")
# input(">>> 로그인 완료 후 엔터 키를 누르세요: ") 
print("🚀 크롤링을 재개합니다...")
# -------------------------------------------------------------

# navigator.webdriver 속성 숨기기 (매우 중요)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    """
})

✅ 드라이버 세션 생성 성공!

🚨🚨 잠시 멈춥니다. 새 창에 Zippoom 로그인 후 콘솔에서 엔터를 눌러주세요. 🚨🚨
🚀 크롤링을 재개합니다...


{'identifier': '2'}

In [ ]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def crawl_zippoom(doro_juso, kapt_name, driver):
    """
    집품(Zippoom) 사이트에서 UI 상호작용을 통해 리뷰를 수집하는 함수
    (Selenium 드라이버 인스턴스를 외부에서 받아 사용합니다.)
    """
    collected_reviews = []
    
    try:
        print(f"\n🔍 검색 시작: {doro_juso} (대상: {kapt_name})")
        search_keyword = doro_juso 
        if not doro_juso:
            search_keyword = kapt_name
            
        driver.get("https://zippoom.com/") # 메인 페이지로 이동
        wait = WebDriverWait(driver, 15)
        actions = ActionChains(driver)

        # =========================================================
        # 1. 검색어 입력 (ActionChains + 투명 버튼 우회)
        # =========================================================
        try:
            # 1. 투명 검색 트리거 찾기
            search_trigger = wait.until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, "input.opacity-0.cursor-pointer")
            ))
            
            # 2. JS로 강제 클릭하여 포커스 활성화
            driver.execute_script("arguments[0].click();", search_trigger)
            time.sleep(1)
            
            
            
            # 3. 입력 및 엔터 (ActionChains 사용)
            (actions
             .send_keys(search_keyword) 
             .pause(0.5)
             .send_keys(Keys.RETURN)
             .perform())
             
            print(f"  👉 [Step 1] 검색어 입력 완료: {search_keyword}")
            
        except Exception as e:
            print(f"  ⚠️ [Step 1] 검색어 입력 실패 (JS/ActionChains 충돌 가능성): {e}")
            return []

        # =========================================================
        # 2. 검색 결과 클릭 (버튼 내부 '도로명' 텍스트 감지)
        # =========================================================
        time.sleep(3) # 검색 결과 로딩 대기
        
        try:
            xpath_result = "//button[.//span[contains(text(), '도로명')]]"
            
            first_result = wait.until(EC.element_to_be_clickable(
                (By.XPATH, xpath_result)
            ))
            
            # JS 강제 클릭
            driver.execute_script("arguments[0].click();", first_result)
            print("  👉 [Step 2] 검색 결과 버튼 클릭 성공")
            
        except Exception as e:
            print(f"  ⚠️ [Step 2] 결과 버튼을 찾을 수 없음 (검색 결과 0건 가능성): {e}")
            return []

        # =========================================================
        # 3. 리뷰 탭 클릭 ('리뷰' 텍스트와 cursor-pointer 클래스)
        # =========================================================
        time.sleep(3) # 상세 페이지 이동 대기
        
        try:
            xpath_tab = "//p[contains(@class, 'cursor-pointer') and contains(., '리뷰')]"
            
            review_tab = wait.until(EC.element_to_be_clickable(
                (By.XPATH, xpath_tab)
            ))
            
            driver.execute_script("arguments[0].click();", review_tab)
            print("  👉 [Step 3] 리뷰 탭 클릭 성공")
            time.sleep(2) # 탭 전환 대기
            
        except Exception as e:
            print(f"  ℹ️ [Step 3] 리뷰 탭 클릭 건너_m: {e}")

        # =========================================================
        # 4. 리뷰 전체 로드 및 데이터 수집
        # =========================================================
        print("  🔄 [Step 4-A] '더보기' 버튼을 반복 클릭하여 모든 리뷰 로드 시작")
        
        # ---------------------------------------------------------
        # [신규 추가] '거주 후기 더보기' 버튼 반복 클릭 로직
        # ---------------------------------------------------------
        while True:
            try:
                # 텍스트 '거주 후기 더보기'를 포함하는 <button> 태그 찾기 (숫자는 동적이므로 포함 여부만 확인)
                # XPath는 하위 요소의 텍스트도 포함해서 찾음 (text() 대신 . 사용)
                more_button_xpath = "//button[contains(., '거주 후기 더보기')]"
                
                # 버튼이 화면에 나타날 때까지 기다림 (최대 5초)
                more_button = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable((By.XPATH, more_button_xpath))
                )
                
                # JS 강제 클릭
                driver.execute_script("arguments[0].click();", more_button)
                time.sleep(1.5) # 새로운 리뷰가 로딩될 시간을 넉넉히 줌
                
            except:
                # 버튼을 찾지 못하거나 클릭할 수 없으면 (모두 로드됨) 루프 종료
                print("  ✅ '더보기' 버튼 반복 클릭 완료. 모든 리뷰가 로드된 것으로 판단.")
                break
        
        # ---------------------------------------------------------
        # [기존] 리뷰 로딩을 위한 최종 스크롤 및 데이터 수집
        # ---------------------------------------------------------
        # 리뷰가 모두 로드된 후, 만약을 대비하여 스크롤
        for _ in range(3):
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(0.5)
            
        print("  👉 [Step 4-B] 로드된 리뷰 블록 분석 및 데이터 수집 시작")

        # 1. 모든 리뷰 블록 찾기 (data-testid='리뷰' 사용)
        review_blocks = driver.find_elements(By.XPATH, "//div[@data-testid='리뷰']")

        if not review_blocks:
            print("  ⚠️ 리뷰 블록을 찾지 못했습니다.")
            return []

        for idx, block in enumerate(review_blocks):
            review_item = {
                'kaptName': kapt_name,
                'doroJuso': doro_juso,
                'Score': None,
                'Pros': None,
                'Cons': None
            }
            
            # --- A. "전체 보기" 클릭하여 리뷰 확장 ---
            try:
                full_view_xpath = ".//p[contains(text(), '전체 보기')]/.." 
                full_view_btn = block.find_element(By.XPATH, full_view_xpath)
                driver.execute_script("arguments[0].click();", full_view_btn)
                time.sleep(0.3) 
            except:
                pass 

            # --- B. 구조화된 데이터 추출 ---
            try:
                review_item['Score'] = block.find_element(By.XPATH, ".//p[contains(@class, 'font-bold')]").text
            except:
                pass 

            try:
                pros_xpath = ".//p[text()='장점']/following-sibling::p[1]"
                review_item['Pros'] = block.find_element(By.XPATH, pros_xpath).text
            except:
                review_item['Pros'] = "" 

            try:
                cons_xpath = ".//p[text()='단점']/following-sibling::p[1]"
                review_item['Cons'] = block.find_element(By.XPATH, cons_xpath).text
            except:
                review_item['Cons'] = "" 
                
            collected_reviews.append(review_item)

        print(f"  ✅ [완료] 총 {len(collected_reviews)}건의 리뷰 구조화 성공")
        return collected_reviews

    except Exception as e:
        print(f"  ❌ [Fatal Error] 프로세스 중단: {e}")
        return []

In [ ]:
import pandas as pd
import time
from collections import defaultdict
# driver 인스턴스와 crawl_zippoom 함수는 정의되어 있다고 가정합니다.

# 결과를 담을 리스트 (defaultdict를 사용하는 것이 효율적이지만, 
# 기존처럼 list of dicts로 유지하여 Pandas로 변환합니다.)
reviews_data = []

# 실행 루프
for idx, row in df.iterrows():
    # crawl_zippoom 함수가 driver 인스턴스를 받도록 수정되었으므로 인자로 전달
    reviews = crawl_zippoom(row['doroJuso'], row['kaptName'], driver)
    
    # [핵심 변경] reviews는 이미 {'kaptName': ..., 'Score': ..., 'Pros': ...} 형태의
    # 딕셔너리 리스트이므로, 그대로 reviews_data에 추가합니다.
    for r in reviews:
        reviews_data.append(r) 
    
    time.sleep(3) # 과부하 방지

# 루프 종료 후 드라이버 종료
driver.quit()

# 결과 확인 및 저장
print(f"\n총 수집된 리뷰: {len(reviews_data)}건")

if reviews_data:
    # list of dicts를 바로 DataFrame으로 변환합니다.
    df_final_reviews = pd.DataFrame(reviews_data)
    
    # (선택) 저장 파일명을 구조화된 데이터임을 반영하여 변경
    df_final_reviews.to_csv('리뷰_구조화_결과.csv', index=False, encoding='utf-8-sig')
    
    print("✅ 리뷰 데이터가 '리뷰_구조화_결과.csv'로 저장되었습니다.")



🔍 검색 시작: 청년주택 와이엔타워 / 서울특별시 노원구 공릉동 동일로 1000

[검색 시도] '서울특별시 노원구 공릉동 동일로 1000' (원본 도로명 주소)
  ✅ 검색창 발견: input[type='text']
  검색 시도 실패 (원본 도로명 주소): Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x604103
	0x604144
	0x40e71d
	0x45a03d
	0x45a41b
	0x4a17f2
	0x47c954
	0x49ee17
	0x47c706
	0x44da30
	0x44ed54
	0x8757b4
	0x87098a
	0x62c392
	0x61c4c8
	0x62324d
	0x60c478
	0x60c63c
	0x5f67ca
	0x76f85d49
	0x778ad6db
	0x778ad661


[검색 시도] '공릉동 동일로 1000' (시/구 제거 주소)
  ✅ 검색창 발견: input[type='text']


KeyboardInterrupt: 